## 1. Carregamento dos Dados

In [ ]:
import pandas as pd

# Carregar o dataset (certifique-se de que o arquivo 'titanic.csv' esteja na mesma pasta do notebook)
df = pd.read_csv('titanic.csv')

# Visualizar as 5 primeiras linhas do dataset
print(df.head())

# Exibir informações gerais do dataset
print(df.info())

## 2. Tratamento e Preparação das Variáveis

Nesta etapa, realizamos o tratamento de valores ausentes, conversão de variáveis categóricas e criação de novas features.

**Variáveis importantes (tradução):**
- PassengerId: ID do Passageiro
- Survived: Sobreviveu
- Pclass: Classe
- Name: Nome
- Sex: Sexo
- Age: Idade
- SibSp: Nº de Irmãos/Espos
- Parch: Nº de Pais/Filhos
- Fare: Tarifa
- Embarked: Ponto de Embarque

In [ ]:
# Verificar a quantidade de valores ausentes em cada coluna
print(df.isnull().sum())

# Preencher valores ausentes na coluna 'Age' com a mediana
df['Age'].fillna(df['Age'].median(), inplace=True)

# Preencher 'Embarked' com a moda (valor que mais aparece)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

# Verificar novamente os valores ausentes
print(df.isnull().sum())

In [ ]:
# Converter a variável 'Sex' para numérico: male -> 0, female -> 1
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

# Verificar as primeiras linhas para confirmar a conversão
print(df[['Sex']].head())

In [ ]:
# Criar a feature 'FamilySize' (Tamanho da Família)
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

# Visualizar a nova feature
print(df[['SibSp', 'Parch', 'FamilySize']].head())

## 3. Divisão dos Dados

Dividimos o dataset em conjuntos de treinamento e teste para avaliar a capacidade de generalização do modelo. Selecionaremos algumas features relevantes e a variável alvo (Sobreviveu).

In [ ]:
from sklearn.model_selection import train_test_split

# Selecionar as features: Classe, Sexo, Idade, Tarifa e FamilySize
X = df[['Pclass', 'Sex', 'Age', 'Fare', 'FamilySize']]
y = df['Survived']

# Dividir os dados (70% treino, 30% teste) com estratificação para manter a proporção de sobreviventes
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print("Tamanho do conjunto de treinamento:", X_train.shape)
print("Tamanho do conjunto de teste:", X_test.shape)

## 4. Treinamento do Modelo de Regressão Logística

Agora, treinamos o modelo de regressão logística utilizando os dados de treinamento.

In [ ]:
from sklearn.linear_model import LogisticRegression

# Instanciar o modelo de Regressão Logística (definindo max_iter para garantir convergência)
modelo = LogisticRegression(max_iter=200)

# Treinar o modelo com os dados de treinamento
modelo.fit(X_train, y_train)

## 5. Exibição e Interpretação dos Coeficientes

Após o treinamento, exibimos os coeficientes (pesos) do modelo, que indicam o impacto de cada variável preditora na probabilidade de sobrevivência.

In [ ]:
# Exibir os coeficientes (pesos) do modelo
print("Pesos (coeficientes):", modelo.coef_)

# Exibir o intercepto
print("Intercepto:", modelo.intercept_)

# Exibir os coeficientes de forma organizada, associando cada feature ao seu respectivo peso
features = X.columns
for feature, coef in zip(features, modelo.coef_[0]):
    print(f"{feature}: {coef}")

## 6. Geração das Curvas de Análise de Erro

Nesta etapa, geramos diversas curvas para analisar o desempenho do modelo no conjunto de teste:

- **Curva ROC:** Taxa de Verdadeiro Positivo (TPR) vs. Taxa de Falso Positivo (FPR).
- **Curva Precision-Recall:** Relação entre Precisão e Recall.
- **Curva de Erro vs. Threshold:** Variação da taxa de erro para diferentes thresholds.
- **Curva de Calibração (Opcional):** Comparação entre probabilidades previstas e probabilidades reais.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, average_precision_score
from sklearn.calibration import calibration_curve

# Obter as probabilidades preditas para a classe positiva
y_prob = modelo.predict_proba(X_test)[:, 1]

## 6.1. Curva ROC

fpr, tpr, thresholds_roc = roc_curve(y_test, y_prob)
auc_value = roc_auc_score(y_test, y_prob)

plt.figure(figsize=(8,6))
plt.plot(fpr, tpr, label=f"Curva ROC (AUC = {auc_value:.2f})")
plt.plot([0, 1], [0, 1], 'k--', label="Linha de referência")
plt.xlabel("Taxa de Falso Positivo (FPR)")
plt.ylabel("Taxa de Verdadeiro Positivo (TPR)")
plt.title("Curva ROC")
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

## 6.2. Curva Precision-Recall

precision, recall, thresholds_pr = precision_recall_curve(y_test, y_prob)
avg_precision = average_precision_score(y_test, y_prob)

plt.figure(figsize=(8,6))
plt.plot(recall, precision, label=f"Curva Precision-Recall (AP = {avg_precision:.2f})")
plt.xlabel("Recall")
plt.ylabel("Precisão")
plt.title("Curva Precision-Recall")
plt.legend(loc="lower left")
plt.grid(True)
plt.show()

## 6.3. Curva de Erro vs. Threshold

thresholds = np.linspace(0, 1, 100)
error_rates = []

for thresh in thresholds:
    y_pred_thresh = (y_prob >= thresh).astype(int)
    error_rate = np.mean(y_pred_thresh != y_test)
    error_rates.append(error_rate)

plt.figure(figsize=(8,6))
plt.plot(thresholds, error_rates, label="Taxa de Erro")
plt.xlabel("Threshold")
plt.ylabel("Taxa de Erro")
plt.title("Curva de Erro vs. Threshold")
plt.legend()
plt.grid(True)
plt.show()

## 6.4. Curva de Calibração (Opcional)

prob_true, prob_pred = calibration_curve(y_test, y_prob, n_bins=10)

plt.figure(figsize=(8,6))
plt.plot(prob_pred, prob_true, marker='o', label="Curva de Calibração")
plt.plot([0, 1], [0, 1], linestyle='--', label="Calibração Perfeita")
plt.xlabel("Probabilidade Prevista")
plt.ylabel("Probabilidade Real (em cada bin)")
plt.title("Curva de Calibração")
plt.legend()
plt.grid(True)
plt.show()